# Exploratory Data Analysis - GitHub repo's issues

In [360]:
import json
import numpy as np
import pandas as pd
import datetime


import warnings
# warnings.filterwarnings('ignore')

# # Suppress pandas's warning
# warnings.simplefilter(action='ignore', category=Warning)

# https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
pd.options.mode.chained_assignment = None  # default='warn'


## Data requirements

Below is a list of interested data that we will use in the initial analysis
* How long the issues open
* First comment response time
* Number of comments
* Number of participants
* Labels associated with each issue
* Milestone

Later will be done in the later part - Find data for each year where
* Top words in question
* Top words in answers
* Top participants + Type and company

## Cleaning data - High-level data

* Extract only interested columns which are
[
'issue_id',
'title',
'contents',
'authorLogin',
'authorAssociation',
'createdAt',
'closedAt',
'closed',
'comments_count',
'comments_data',
'participants_count',
'labels',
'milestone'
]

* Cast datetime columns to ['createdAt', 'closedAt'] datetime 
* Create a new column 'closedDuration_days' to store a timediff between the columns ['closedAt', 'createdAt'] in days
* Additional columns for comments data
  * Create a new column 'firstCommentCreatedAt' to store a datetime object the first comment is created
  * Create a new column 'firstCommentDuration_days' to store a timediff between the columns 'createdAt' and the creation time of the first comment in days
  * New columns for the first comment author info ['firstCommentAuthor', 'firstCommentAuthorAssociation'] 
  * New columns for total number of unique comment authors based on authorAssociation ['num_unique_comment_author_MEMBER', 'num_unique_comment_author_CONTRIBUTOR', 'num_unique_comment_author_COLLABORATOR', 'num_unique_comment_author_NONE']      
* Extract 'milestone' title and replace the 'milestone' column with this value
* Perform one-hot encoding to the label column

In [404]:
def get_comment_info_columns(row):
    '''
    Function to extract data from a dataframe's row that will be used in the function 'create_highlevel_df'
    
    Input argument:
        row: A row of data frame containing github issue info
        
    Output argument:
        a list containing the following information
            [
                'firstCommentCreatedAt', 
                'firstCommentAuthor', 
                'firstCommentAuthorAssociation',
                'num_unique_comment_author_MEMBER',
                'num_unique_comment_author_CONTRIBUTOR',
                'num_unique_comment_author_COLLABORATOR',
                'num_unique_comment_author_NONE'
            ]
        
    '''
    
    # If the comments_count is greater than 0
    if row['comments_count'] > 0:
        
        # --- Extract data for 'firstCommentCreatedAt', 'firstCommentAuthor', 'firstCommentAuthorAssociation'
        
        out = [
            row['comments_data'][0]['createdAt'],
            row['comments_data'][0]['authorLogin'], 
            row['comments_data'][0]['authorAssociation']
        ]
        
        # --- Extract data for 'num_unique_comment_*'
        # We need to extract data from the 'comments_data' column
        
        # Get a list of all comment authors first
        list_comment_authors = []
        for cur_comment in row['comments_data']:
            cur_author = {
                'login': cur_comment['authorLogin'],
                'association': cur_comment['authorAssociation']
            }
            list_comment_authors.append(cur_author)
            
        unique_comment_authors = [dict(y) for y in set(tuple(x.items()) for x in list_comment_authors)]
        
        # Then, get count for each type
        dict_author_assoc = {
            'MEMBER': 0,
            'CONTRIBUTOR': 0,
            'COLLABORATOR': 0,
            'NONE': 0
        }
        
        for cur_comment_author in unique_comment_authors:
            cur_assoc = cur_comment_author['association']
            dict_author_assoc[cur_assoc] += 1
            
        # Append data to the output list
        out = out + [
            dict_author_assoc['MEMBER'],
            dict_author_assoc['CONTRIBUTOR'],
            dict_author_assoc['COLLABORATOR'],
            dict_author_assoc['NONE']
        ]
    else:
        # Otherwise, output a default output list
        out = [None, None, None, 0, 0, 0, 0]

        
    return out

In [405]:
def create_highlevel_df(json_path):
    '''
    Extract github issues data from the specified path and output an extracted summary data as a new dataframe
    '''
    
    # Load json data and create dataframe
    with open(json_path) as json_file:  
        data_raw = json.load(json_file)

    df_raw = pd.DataFrame.from_dict(data_raw)
    
    # Let's get only columns that we need for the analysis now
    new_cols = [
        'issue_id',
        'title',
        'contents',
        'authorLogin',
        'authorAssociation',
        'createdAt',
        'closedAt',
        'closed',
        'comments_count',
        'participants_count',
        'comments_data',
        'labels',
        'milestone'
     ]
    
    df_out = df_raw[new_cols]
    
    # --- Cast columns to datetime
    df_out[['createdAt', 'closedAt']] = df_out[['createdAt', 'closedAt']].apply(pd.to_datetime)
    
    # --- Find closedDuration_days
    get_diff_days = lambda s: (s.dt.total_seconds() / (24 * 60 * 60)).round(2)
#     df_out['closedDuration_days'] = ((df_out['closedAt'] - df_out['createdAt']).dt.total_seconds() / (24 * 60 * 60)).round(2)
    df_out['closedDuration_days'] = get_diff_days(df_out['closedAt'] - df_out['createdAt'])

    # --- Find comment info
    cols_first_comments = [
        'firstCommentCreatedAt', 
        'firstCommentAuthor', 
        'firstCommentAuthorAssociation',
        'num_unique_comment_author_MEMBER',
        'num_unique_comment_author_CONTRIBUTOR',
        'num_unique_comment_author_COLLABORATOR',
        'num_unique_comment_author_NONE'
    ]
    df_out[cols_first_comments] = pd.DataFrame(
                    df_out.apply(lambda row: get_comment_info_columns(row), axis=1).values.tolist(), 
                     index= df_out.index)

    df_out['firstCommentCreatedAt'] = pd.to_datetime(df_out['firstCommentCreatedAt'])
    df_out['firstCommentDuration_days'] = get_diff_days(df_out['firstCommentCreatedAt'] - df_out['createdAt'])
    
    # --- Extract 'milestone' title and replace the 'milestone' colummn with this value
    df_out['milestone'] = df_out['milestone'].apply(lambda x: x['title'] if x else '')
    
    # --- Perform one hot encoding for the labels columns
    
    # Get a list of labels
    list_labels = []
    for index, row in df_out.iterrows():
        list_labels = list_labels + row['labels']

    list_labels = list(set(list_labels))
    
    # Then, create new columns for each of those labels
    
    # Create a dictionary to map those labels with a boolean value for each record in the dataset
    dict_labels = dict((el,[]) for el in list_labels)
    for index, row in df_out.iterrows():
        for cur_label in list_labels:
            cur_bool = False
            if cur_label in row['labels']:
                cur_bool = True

            dict_labels[cur_label].append(cur_bool)
    
    # Then, create a dataframe for it and merge the newly created dataframe with the current output dataframe
    df_out = pd.concat([df_out, pd.DataFrame.from_dict(dict_labels)], axis=1)
    
    # Finally, prepend those newly added label columns with 'Label_'
    dict_rename = {}
    for cur_label in list_labels:
        dict_rename[cur_label] = 'label_' + cur_label
        
    df_out = df_out.rename(columns=dict_rename)

    
    # --- Reorder columns
    final_cols = [
        'issue_id',
        'title',
        'contents',
        'authorLogin',
        'authorAssociation',
        'createdAt',
        'closed',
        'closedAt',        
        'closedDuration_days',
        'milestone',
        'participants_count',
        'comments_count',
        'firstCommentCreatedAt',
        'firstCommentDuration_days',
        'firstCommentAuthor',
        'firstCommentAuthorAssociation',
        'num_unique_comment_author_MEMBER',
        'num_unique_comment_author_CONTRIBUTOR',
        'num_unique_comment_author_COLLABORATOR',
        'num_unique_comment_author_NONE',
        'labels'
    ]
    
    # Then, append the one hot encoding's label columns
    final_cols = final_cols + ['label_' + cur_label for cur_label in list_labels]
    df_out = df_out[final_cols]
    
    return df_out



In [403]:
list_libs = [
    'qunit',
    'mocha',
    'jest',
    'jasmine',
    'funcunit',
    'puppeteer',
    'cypress'
]

for cur_lib in list_libs:
    print('***** [{}] START repo#{}" *****'.format(str(datetime.datetime.now()), cur_lib))
        
    json_path = '../data/github_repo_issues_{}.json'.format(cur_lib)
    df_cur = create_highlevel_df(json_path) 
    
    print('|-- Total records: ', df_cur.shape[0])
    
    # Save data to csv
    csv_filename = 'temp/repo_issue_summary_{}.csv'.format(cur_lib)
    print('|-- Save data to "{}"...'.format(csv_filename))
    
    df_cur.to_csv(csv_filename, index=False)

***** [2019-07-06 12:44:23.277969] START repo#qunit" *****
|-- Total records:  708
|-- Save data to "temp/repo_issue_summary_qunit.csv"...
***** [2019-07-06 12:44:23.871952] START repo#mocha" *****
|-- Total records:  2356
|-- Save data to "temp/repo_issue_summary_mocha.csv"...
***** [2019-07-06 12:44:26.126188] START repo#jest" *****
|-- Total records:  4530
|-- Save data to "temp/repo_issue_summary_jest.csv"...
***** [2019-07-06 12:44:30.063584] START repo#jasmine" *****
|-- Total records:  1198
|-- Save data to "temp/repo_issue_summary_jasmine.csv"...
***** [2019-07-06 12:44:30.868725] START repo#funcunit" *****
|-- Total records:  154
|-- Save data to "temp/repo_issue_summary_funcunit.csv"...
***** [2019-07-06 12:44:30.985343] START repo#puppeteer" *****
|-- Total records:  3067
|-- Save data to "temp/repo_issue_summary_puppeteer.csv"...
***** [2019-07-06 12:44:32.703312] START repo#cypress" *****
|-- Total records:  3541
|-- Save data to "temp/repo_issue_summary_cypress.csv"...
